In [5]:
import pandas as pd
import numpy as np

import time
from datetime import datetime
from datetime import timedelta

import os

In [6]:
current_folder = os.path.join(os.getcwd(),'data/stocks')

In [7]:
files_list = os.listdir(current_folder)

In [8]:
data = pd.DataFrame()
for file in files_list:
    current_file = pd.read_csv(os.path.join(current_folder,file), encoding="latin-1")
    file_name = file[:file.find('_(')]

    current_file.drop(['Waluta'], axis = 1, inplace = True)
    try:
        data = pd.merge(data, current_file, left_on = 'Data',  right_on = 'Data', suffixes=('', file_name))
    except:
        data = current_file
    

In [9]:
columns = []
for column in data.columns:
    columns.append(column.replace('Wartoæ dla ', '').replace('¿','ż'))

In [10]:
data.columns=columns

In [11]:
backup_data = data

In [12]:
data = backup_data

data

In [13]:
origin_columns = data.columns

In [14]:
on = origin_columns[1:]

In [15]:
base_names = []

In [16]:
for column in on:
    base_names.append(column)
    name = column[:column.find(" (")]
    
    # Obliczamy cumulative sum
    data[name+" "+'Suma'] = data[column].cumsum()

    # Obliczamy kroczące średnie dla 2, 5, 10 i 30 dni.
    data[name+" "+'Krocząca Średnia [2d]']   = data[column].rolling(window=2).mean()
    data[name+" "+'Krocząca Średnia [5d]']   = data[column].rolling(window=5).mean()
    data[name+" "+'Krocząca Średnia [10d]']  = data[column].rolling(window=10).mean()
    data[name+" "+'Krocząca Średnia [30d]']  = data[column].rolling(window=30).mean()
    data[name+" "+'Krocząca Średnia [60d]']  = data[column].rolling(window=60).mean()
    data[name+" "+'Krocząca Średnia [120d]']  = data[column].rolling(window=120).mean()
    data[name+" "+'Krocząca Średnia [360d]']  = data[column].rolling(window=360).mean()

    data[name+" "+'Pierwsza pochodna'] = data[column].diff()
    data[name+" "+'Druga pochodna'] = data[column].diff().diff()

In [17]:
data

,Data,Goldman Sachs Japonia (K),Goldman Sachs Zrównoważony (A),Goldman Sachs Stabilny Globalnej Dywersyfikacji (A),Goldman Sachs Konserwatywny (A),Goldman Sachs Japonia Suma,Goldman Sachs Japonia Krocząca Średnia [2d],Goldman Sachs Japonia Krocząca Średnia [5d],Goldman Sachs Japonia Krocząca Średnia [10d],Goldman Sachs Japonia Krocząca Średnia [30d],...,Goldman Sachs Konserwatywny Suma,Goldman Sachs Konserwatywny Krocząca Średnia [2d],Goldman Sachs Konserwatywny Krocząca Średnia [5d],Goldman Sachs Konserwatywny Krocząca Średnia [10d],Goldman Sachs Konserwatywny Krocząca Średnia [30d],Goldman Sachs Konserwatywny Krocząca Średnia [60d],Goldman Sachs Konserwatywny Krocząca Średnia [120d],Goldman Sachs Konserwatywny Krocząca Średnia [360d],Goldman Sachs Konserwatywny Pierwsza pochodna,Goldman Sachs Konserwatywny Druga pochodna
0,2014-03-24,140.33,278.99,100.00,250.78,140.33,NaN,NaN,NaN,NaN,...,250.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-03-25,140.38,280.70,100.28,250.79,280.71,140.355,NaN,NaN,NaN,...,501.57,250.785,NaN,NaN,NaN,NaN,NaN,NaN,0.01,NaN
2,2014-03-26,141.59,282.48,100.25,250.80,422.30,140.985,NaN,NaN,NaN,...,752.37,250.795,NaN,NaN,NaN,NaN,NaN,NaN,0.01,2.842171e-14
3,2014-03-27,142.79,282.57,100.49,250.77,565.09,142.190,NaN,NaN,NaN,...,1003.14,250.785,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,-4.000000e-02
4,2014-03-28,143.98,283.37,100.63,250.82,709.07,143.385,141.814,NaN,NaN,...,1253.96,250.795,250.792,NaN,NaN,NaN,NaN,NaN,0.05,8.000000e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,2024-05-06,386.37,450.90,137.31,310.13,543040.37,386.400,385.602,382.289,388.460333,...,682590.79,310.050,309.912,309.670,309.053667,308.064667,305.519417,294.959083,0.16,7.000000e-02
2530,2024-05-07,390.50,450.48,137.71,310.05,543430.87,388.435,387.180,383.639,388.451667,...,682900.84,310.090,309.974,309.755,309.120667,308.132167,305.602167,295.057444,-0.08,-2.400000e-01
2531,2024-05-08,386.59,451.51,137.56,310.07,543817.46,388.545,387.986,384.445,387.984667,...,683210.91,310.060,310.020,309.828,309.185333,308.198833,305.684333,295.155917,0.02,1.000000e-01
2532,2024-05-09,388.34,452.61,137.68,310.13,544205.80,387.465,387.646,385.262,387.610667,...,683521.04,310.100,310.070,309.898,309.251000,308.265000,305.767500,295.254417,0.06,4.000000e-02


In [14]:
current_folder = os.path.join(os.getcwd(),'data')
data.to_csv(os.path.join(current_folder,'data.csv'))

In [18]:
base_names

['Goldman Sachs Japonia (K)',
 'Goldman Sachs Zrównoważony (A)',
 'Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)',
 'Goldman Sachs Konserwatywny (A)']

In [16]:
import json
with open(os.path.join(current_folder,'names.json'), 'w') as f:
    json.dump(base_names, f, indent=4, sort_keys=True)
    print("Saved")

Saved


In [17]:
data.columns

Index(['Data', 'Goldman Sachs Japonia (K)', 'Goldman Sachs Zrównoważony (A)',
       'Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)',
       'Goldman Sachs Konserwatywny (A)', 'Goldman Sachs Japonia Suma',
       'Goldman Sachs Japonia Krocząca Średnia [2d]',
       'Goldman Sachs Japonia Krocząca Średnia [5d]',
       'Goldman Sachs Japonia Krocząca Średnia [10d]',
       'Goldman Sachs Japonia Krocząca Średnia [30d]',
       'Goldman Sachs Japonia Krocząca Średnia [60d]',
       'Goldman Sachs Japonia Krocząca Średnia [120d]',
       'Goldman Sachs Japonia Krocząca Średnia [360d]',
       'Goldman Sachs Japonia Pierwsza pochodna',
       'Goldman Sachs Japonia Druga pochodna',
       'Goldman Sachs Zrównoważony Suma',
       'Goldman Sachs Zrównoważony Krocząca Średnia [2d]',
       'Goldman Sachs Zrównoważony Krocząca Średnia [5d]',
       'Goldman Sachs Zrównoważony Krocząca Średnia [10d]',
       'Goldman Sachs Zrównoważony Krocząca Średnia [30d]',
       'Goldman Sachs Zrów

In [31]:
[column for column in data.columns if (column.find('Goldman Sachs Japonia') == 0)]

['Goldman Sachs Japonia (K)',
 'Goldman Sachs Japonia Suma',
 'Goldman Sachs Japonia Krocząca Średnia [2d]',
 'Goldman Sachs Japonia Krocząca Średnia [5d]',
 'Goldman Sachs Japonia Krocząca Średnia [10d]',
 'Goldman Sachs Japonia Krocząca Średnia [30d]',
 'Goldman Sachs Japonia Krocząca Średnia [60d]',
 'Goldman Sachs Japonia Krocząca Średnia [120d]',
 'Goldman Sachs Japonia Krocząca Średnia [360d]',
 'Goldman Sachs Japonia Pierwsza pochodna',
 'Goldman Sachs Japonia Druga pochodna']

In [29]:
data.columns[1].find('Goldman Sachs Japonia (K)') == 0

True

In [28]:
data.columns[1]

'Goldman Sachs Japonia (K)'

In [30]:
series_names = [{'label': name, 'value':name} for name in base_names]

In [36]:
[ value['value']  for value in series_names]

['Goldman Sachs Japonia (K)',
 'Goldman Sachs Zrównoważony (A)',
 'Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)',
 'Goldman Sachs Konserwatywny (A)']

In [36]:
[column for column in series_names if (column['value'].find('Goldman Sachs Japonia') == 0)]

[{'label': 'Goldman Sachs Japonia (K)', 'value': 'Goldman Sachs Japonia (K)'}]

In [1]:
import linear

In [3]:
linear_output = linear.get_linear_predictions(future_days = 30)

In [4]:
linear_output

{'Goldman Sachs Japonia (K)': 288.9850552311913,
 'Goldman Sachs Zrównoważony (A)': 386.26714687709307,
 'Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)': 131.15891342355675,
 'Goldman Sachs Konserwatywny (A)': 289.3989206111519,
 'Goldman Sachs Japonia Suma': 508956.93065944,
 'Goldman Sachs Japonia Krocząca Średnia [2d]': 289.2097103537965,
 'Goldman Sachs Japonia Krocząca Średnia [5d]': 288.1969948173868,
 'Goldman Sachs Japonia Krocząca Średnia [10d]': 287.8161577012261,
 'Goldman Sachs Japonia Krocząca Średnia [30d]': 285.84611351628376,
 'Goldman Sachs Japonia Krocząca Średnia [60d]': 282.4542209108664,
 'Goldman Sachs Japonia Krocząca Średnia [120d]': 276.05948760557817,
 'Goldman Sachs Japonia Krocząca Średnia [360d]': 259.0740907348759,
 'Goldman Sachs Japonia Pierwsza pochodna': 0.27342616930851926,
 'Goldman Sachs Japonia Druga pochodna': 0.12458384737023404,
 'Goldman Sachs Zrównoważony Suma': 822450.6593673046,
 'Goldman Sachs Zrównoważony Krocząca Średnia [2d]': 385.

In [ ]:
    """
    daq.Gauge(
        showCurrentValue=True,
        units="PLN",
        value=5,
        label='Default',
        max=10,
        min=0
    )
    """

In [19]:
for column in data.columns[:5]:
    print(column)

    linear_output[column]
    
    

Data
Goldman Sachs Japonia (K)
Goldman Sachs Zrównoważony (A)
Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)
Goldman Sachs Konserwatywny (A)


In [28]:
data.tail(1)['Goldman Sachs Japonia (K)'].values[0]

389.35

In [25]:
data.min()

Data                                                                       2014-03-24
Goldman Sachs Japonia (K)                                                      138.76
Goldman Sachs Zrównoważony (A)                                                 264.59
Goldman Sachs Stabilny Globalnej Dywersyfikacji (A)                             99.96
Goldman Sachs Konserwatywny (A)                                                250.77
Goldman Sachs Japonia Suma                                                     140.33
Goldman Sachs Japonia Krocząca Średnia [2d]                                    138.82
Goldman Sachs Japonia Krocząca Średnia [5d]                                   139.486
Goldman Sachs Japonia Krocząca Średnia [10d]                                  141.225
Goldman Sachs Japonia Krocząca Średnia [30d]                               142.734667
Goldman Sachs Japonia Krocząca Średnia [60d]                                 145.6025
Goldman Sachs Japonia Krocząca Średnia [120d]         